### Center of Neuroengineering and Therapeutics
# IEEG Pre-processing Pipeline
This notebook provides a recommended usage pipeline for the iEEG pre-processing toolkit.

## Imports



In [1]:
# set search path
import os,sys
import pandas as pd
import numpy as np

!pip install git+https://github.com/ieeg-portal/ieegpy.git # Install ieegpy toolbox directly from github
from ieeg.auth import Session

!pip install git+https://github.com/haoershi/CNT_research_tools.git#subdirectory=python
from CNTtools import iEEGPreprocess

path = os.getcwd()

## Pipeline implementation

In [ ]:
#@title Initialize Session
#@markdown
#@markdown Initialize iEEGPreprocess session.
#@markdown 
#@markdown Suggested syntax:
#@markdown
#@markdown ```session = iEEGPreprocess()```
session = iEEGPreprocess()

### Available Methods 
**Arguments:**
* datasets: diction of data objects
* meta: metadata as a pandas dataframe with columns | filename | start | stop | dura | fs |
* num_data: number of data objects available
* users: list of available iEEG login usernames
* user: dict, user information of the active account
* user_data_dir: str, default path to save user data
**Methods:**
* download_data: download data from ieeg.org
* list_data: list meta info
* load_data: load data from saved pickle files
* login: login with default or specified username
* login_config: input username and password info to create config files
* save: save session as pickle file

In [ ]:
dir(session)

In [ ]:
#@title Login Configuration
#@markdown Set up iEEG.org login info for obtaining data
#@markdown
#@markdown Initialize iEEGPreprocess session.
#@markdown 
#@markdown Suggested syntax:
#@markdown
#@markdown ```session.login_config()```
#@markdown 
session.login_config()

In [ ]:
#@title Get Data via Input
#@markdown Specify the filename, start/stop time, electrodes, etc.
iEEG_filename = "HUP172_phaseII"#@param {type:"string"}
start_time = 402580 #@param {type:"number"}
stop_time = 402600 #@param {type:"number"}
electrodes = "LE10, LE11, LH01, LH02, LH03, LH04" #@param {type:"string"}
electrodes = electrodes.split(', ')
# param = {'filename':iEEG_filename,
#          'start':start_time,
#          'stop':stop_time,
#          'electrodes':electrodes}

In [ ]:
#@title Get Data via File
#@markdown Specify the path to file containing data info (.csv/.tsv)
#@markdown Format Filename | Start time | Stop time | Electrodes
file_path = "test/getData_testInput.csv" # @param {type:"string"}
param = pd.read_csv(file_path)

#tmp fix
param = param.iloc[0,:3].to_dict()

In [ ]:
#@title Import data
#@markdown Download data from ieeg.org.
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data = session.download_data(filename, start_time, stop_time)```
data = session.download_data(iEEG_filename, start_time, stop_time, select_elecs = electrodes)
session.meta

### Available Methods for Data Object
**Arguments:**
* filename/start/stop: inputs
* select_elects: selected channels for downloading
* ignore_elecs: channels excluded from downloading
* dura: duration of data segment
* data: active data array after pre-processing
* fs: sampling frequency
* ch_names: active channel names after pre-processing
* ref_chnames: channel names after re-referencing
* index: dataset index in current session
* conn: diction storing connectivity data
* history: operations done on data object
* raw: raw data array
* raw_chs: raw channel names
* username: ieeg username
* user_data_dir: str, default path to save user data
* nonieeg: boolean mask for non-intracranial channels
* bad: boolean mask for channels containing artifacts
* reject_details: reasons for identify bad channels

**Methods:**

Channels:
* clean_labels: standardize channel names
* find_nonieeg: identify non-iEEG channels
* find_bad_chs: identify bad channels containing artifacts
* reject_nonieeg: identify and remove non-iEEG channels
* reject_artifact: identify and remove artifact channels

Filtering:
* bandpass_filter: apply bandpass filter
* notch_filter: apply notch filter

Re-referencing:
* car: common average re-referencing
* bipolar: bipolar re-referencing
* laplacian: laplacian re-referencing

Connectivity:
* connectivity: calculate various connectivity
* coherence: calculate coherence
* cross_corr: calculate cross correlation
* pearson: calculate pearson correlation
* plv: calculate phase-locking value
* relative_entropy: calculate relative entropy
* squared_pearson: calculate squared pearson correlation

Calculate features:
* bandpower: calculate bandpower
* line_length: calculate linelength

Others:
* load_locs: load electrode location information from .csv file
* pre_whiten: pre-whitening of iEEG data
* plot: plot data
* reverse: reverse operation by one step
* save: save session as pickle file

In [ ]:
#@title Standardize channel names
#@markdown Convert channel names to standardized format
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.clean_labels()```
#@markdown
#@markdown Example changes:
#@markdown 'EKG01' -> 'EKG1'

# show cleaned and original labels
data.clean_labels()

No channels changed.


In [ ]:
#@title Identifiy Non-iEEG channels
#@markdown Find and return a boolean mask for Non-iEEG channels, 0 = iEEG, 1 = Non-iEEG
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.find_nonieeg()```
#@markdown ```data.nonieeg```

data.find_nonieeg()
print('Non-iEEG channels includes:')
print(data.ch_names[data.nonieeg])

Non-iEEG channels includes:
['EKG1' 'EKG2']


In [ ]:
#@title Identifiy bad channels
#@markdown Find and return a boolean mask for bad channels (1 = bad channels), and details for reasons to reject.
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.find_bad_chs()```
#@markdown ```data.bad, data.reject_details```

data.find_bad_chs()
print('Bad channels include:')
print(data.ch_names[data.bad])
#@markdown Details contain potential reasons for being a bad channel and corresponding channel indices.
print('\n')
print('Specific reasons for being rejected:')
for k,v in data.reject_details.items():
  print(k,data['ch_names'][v])

Bad channels include:
['LB12' 'RE8' 'RE9' 'RE10']


Specific reasons for being rejected:
noisy ['LB12' 'RE8' 'RE9' 'RE10']
nans []
zeros []
var []
higher_std []
high_voltage []


In [ ]:
#@title Clean data
#@markdown
#@markdown Reject Non-iEEG channels.
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.reject_nonieeg()```
#@markdown
#@markdown Reject bad hannels containing artifacts
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.reject_artifact()```

data.reject_nonieeg()
data.reject_artifact()


In [ ]:
#@title Filtering
#@markdown Filter iEEG signal with bandpass and notch filter.
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```data.bandpass_filter(low_freq,high_freq)```
#@markdown
#@markdown ```data.notch_filter(notch_freq)```
#@markdown
#@markdown ```data.filter(low_freq,high_freq,notch_freq)```
low_freq = 1 # @param {type:"number"}
high_freq = 120 # @param {type:"number"}
notch_freq = 60 # @param {type:"number"}

data.bandpass_filter(low_freq,high_freq)
data.notch_filter(notch_freq)

In [ ]:
#@title Re-referencing
#@markdown Re-reference data to a different montage
#@markdown
#@markdown Available options:
#@markdown * Common Average Re-referencing (CAR)
#@markdown * Bipolar Re-referencing (BR)
#@markdown * Laplacian Re-referencing (LAR): Note, requires location of electrodes and radius threshold
#@markdown
#@markdown Suggested syntax:
#@markdown * CAR: ```data.car()```
#@markdown * BR: ```data.bipolar()```
#@markdown * LAR: ```data.laplacian(loc_file)```
#@markdown * ```data.reref(method)```

ref = "CAR" # @param ["CAR", "BR", "LAR"]
loc_file = 'elec_locs.csv' # @param {type:"string"}
radius = 20 # @param {type:"number"}

if ref == 'CAR':
  data.car()
elif ref == 'BR':
  data.bipolar()
elif ref == 'LAR':
  data.laplacian(locs = loc_file,radius=radius)

In [ ]:
#@title Plot data
#@markdown Generate plot of time-series data
#@markdown 
#@markdown Suggested syntax:
#@markdown 
#@markdown ```fig = data.plot()```
fig = data.plot()

In [ ]:
#@title Connectivity Analysis
#@markdown Connectivity analysis using various methods.
#@markdown
#@markdown Available options:
#@markdown * Pearson Correlation:  ```data.pearson()```
#@markdown * Squared Pearson:  ```data.squared_pearson()```
#@markdown * Cross Correlation:  ```data.cross_corr()```
#@markdown * Coherence:  ```data.coh()```
#@markdown * Phase-Locking Value (PLV):  ```data.plv()```
#@markdown * Relative Entropy:  ```data.relative_entropy()```
#@markdown
#@markdown Optional syntax:
#@markdown
#@markdown ```data.connectivity(method_list)```

conn_method = "pearson" # @param ["pearson", "squared_pearson", "cross_corr", "coh", "plv", "relative_entropy"]

data.connectivity([conn_method])

In [ ]:
#@title Connectivity heatmap
#@markdown Heatmap plot of calculated connectivity matrix.
#@markdown
#@markdown Suggested syntax:
#@markdown 
#@markdown ```fig = data.conn_heatmap(method,frequency)```

conn_method = "pearson" # @param ["pearson", "squared_pearson", "cross_corr", "coh", "plv", "relative_entropy"]
freq_band = "broad" # @param ['delta','theta','alpha','beta','gamma','ripple','broad']
data.conn_heatmap(conn_method,freq_band)

In [ ]:
#@title Save data
#@markdown Save data in pickle format
#@markdown
#@markdown Suggested syntax:
#@markdown 
#@markdown ```session.save(filename)```
#@markdown 
#@markdown ```data.save()```

filename = 'session0'  #@param {type:"string"}
session.save(filename)
data.save()

In [ ]:
#@title Load data
#@markdown Load saved pickle format data
#@markdown
#@markdown Suggested syntax:
#@markdown
#@markdown ```session.load_data(dir)```
#@markdown
#@markdown ```session.load_data(filename)```
session.load_data('')
session.list_data()

## Batch Processing
Code for parallel processing of multiple data files

In [ ]:
#@title Settings
#@markdown Get Data via File:
#@markdown
#@markdown Specify the path to file containing data info (.csv/.tsv)
#@markdown Format Filename | Start time | Stop time | Electrodes
file_path = "test/getData_testInput.csv" # @param {type:"string"}

#@markdown Filtering Settings:

low_freq = 1 # @param {type:"number"}
high_freq = 120 # @param {type:"number"}
notch_freq = 60 # @param {type:"number"}

#@markdown Re-ref method Setting:
ref = "CAR" # @param ["CAR", "BR", "LAR"]
#@markdown Laplacian Setting:
loc_file = 'elec_locs.csv' # @param {type:"string"}
radius = 20 # @param {type:"number"}

#@markdown Connectivity Setting:
conn_method = "pearson" # @param ["pearson", "squared_pearson", "cross_corr", "coh", "plv", "relative_entropy"]

#@markdown Save data:
save = True # @param {type:"boolean"}

In [ ]:
#@title Implementation

param = pd.read_csv(file_path)
param = param.iloc[:2,:3]
param_dict = []
for i in range(param.shape[0]):
  param_dict.append(param.iloc[i,:].to_dict())
session = iEEGPreprocess()
session.login()
for i in range(len(param_dict)):
  param = param_dict[i]
  data = session.download_data(param['filename'], param['start'], param['stop'])
  data.clean_labels()
  data.reject_nonieeg()
  data.reject_artifact()
  data.filter(low_freq,high_freq,notch_freq)
  data.reref(ref,locs=loc_file,radius=radius)
  data.connectivity([conn_method])
  if save:
    data.save()
